In [1]:
import pandas as pd
import os
from datetime import datetime
from item_parser import parse
import sql_con

In [2]:
plytix_prods = pd.read_sql_query("""
    SELECT
        [Артикул],
        [Наименование], 
        [Brand]
    FROM [datahouse].[dbo].[View_Price1C_Auto]""", sql_con.engine_for_upload())
rhm_prods = pd.read_sql_query("""
   SELECT [sku]
      ,[label]
      ,[brand]
  FROM [datahouse].[dbo].[plytix_products]""", sql_con.engine_for_upload())
rhm_prods.columns = ['Артикул'] + list(rhm_prods.columns[1:])
merged_data = pd.merge(rhm_prods, plytix_prods, on='Артикул', how='inner')
merged_data.drop_duplicates(subset='Артикул', inplace=True)
merged_data.drop('Brand', axis=1, inplace=True)
merged_data.drop('label', axis=1, inplace=True)
merged_data = merged_data.reset_index(drop=True)

In [3]:
merged_data[merged_data['Наименование']=='Nutricurls Waves Молочко-спрей легкое питательное несмываемое для кудрявых и вью']

,Артикул,brand,Наименование
4245,99240039299,Wella Professionals,Nutricurls Waves Молочко-спрей легкое питатель...


In [18]:
counter = 0
for item in merged_data["Наименование"][3500:]:
    errors_counter = 0
    while True:
        try:
            parse(item, clear = counter % 100 == 0)
            break
        except Exception as e:
            if errors_counter == 2:
                print(f'Два раза парсинг упал с ошибкой {item}')
                break
            errors_counter += 1
    counter += 1
    print(counter)

1
2


KeyboardInterrupt: 

In [ ]:
counter

In [ ]:
merged_data.loc[merged_data['Наименование']=="DS Volume Mousse мусс для укладки сильной фиксации 300 мл"]

In [3]:
path_to_dir = r'C:/Users/admin/PycharmProjects/pythonProject/price_parsers/parsers_output/BT_parser/'
all_files = os.listdir(path_to_dir)
error_counter = 0
all_pages = []
for index, filename in enumerate(all_files):
    try: 
        all_pages.append(pd.read_excel(path_to_dir+filename))
    except Exception as e:
        raise(e)
        error_counter += 1

data = pd.concat(all_pages).drop(['Unnamed: 0'], axis=1)#.to_excel(f'{str(datetime.today().strftime("%Y-%m-%d"))}.xlsx',index=False)

In [9]:
merged_data.columns = list(merged_data.columns)[:-1]+['Название товара наше']

In [10]:
data['Цена'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if cell_val is not None else None for cell_val in
                data['Цена']]
data['Цена со скидкой'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if not pd.isna(cell_val) else None for
                           cell_val in data['Цена со скидкой']]

In [13]:
new_data = pd.merge(data, merged_data, on='Название товара наше')
new_data.drop('Бренд', axis=1)
new_data.to_excel(rf'C:\Users\admin\PycharmProjects\pythonProject\price_parsers\parsers_output\final_result\{str(datetime.today().strftime("%Y-%m-%d"))}.xlsx')

In [15]:
set_1 = set(list(new_data['Название товара наше']))

In [20]:
set_2 = set(list(pd.read_excel(r'C:\Users\admin\PycharmProjects\pythonProject\price_parsers\wb_parser\final_data.xlsx')['Наименование']))
set_3 = set(list(pd.read_excel(r'C:\Users\admin\PycharmProjects\pythonProject\price_parsers\analysis\final_data_gallery_cosm.xlsx')['Название товара наше']))

In [ ]:
len(set_2)

In [16]:
len(set_1)

1912

In [21]:
len(set_1.union(set_2).union(set_3))

3683

In [24]:
new_data.drop(['Бренд', 'Есть в наличии'], axis=1).to_excel('final_result_bt.xlsx')

In [25]:
merged_data

,Артикул,brand,Название товара наше
0,/0,Sim Sensitive,SensiDO Cream Color 3 in 1 краска для волос /0
1,/16,Sim Sensitive,SensiDO Magic паста для цветного осветления во...
2,/3,Sim Sensitive,SensiDO Cream Color 3 in 1 краска для волос /3
3,/34,Sim Sensitive,SensiDO Magic паста для цветного осветления во...
4,/4,Sim Sensitive,SensiDO Cream Color 3 in 1 краска для волос /4
...,...,...,...
5066,hc0004,Mr.Natty,Mr.Natty Pomade Wax Hair Preparation / Воск дл...
5067,natty1,Mr.Natty,Mr.Natty Matte Texture Powder / Пудра для объе...
5068,seasalt,Mr.Natty,Mr.Natty Triple Sea Salt Spray / Морская соль ...
5069,sht1,LiHead,LiHead Штатив Напольный для манекена
